# write issues here:

can not distinguish that mapk is a gene not mask: add a guidance to the system


# take actions:
1) pseudo_sentence -> id -> NetMedGPT -> list.csv
2) run on NetMedGPT


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from convertor import *
from helpers import *
from sentence_preprocessing import *

/home/bbc8731/miniconda3/envs/netmedgpt_llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# read data from here
nodes = pd.read_csv('data/nodes_snake.csv', sep= ',')
emb_nodes = pd.read_csv("data/emb_pubmedbert_all_nodes.csv", index_col=0)
edges = pd.read_csv('data/edges.csv', sep= ',')
relation_index = edges[['relation', 'z_index']].drop_duplicates()
mask_token = edges['z_index'].max() + 1
examples_a = ["for diabetes with egft mutation what is the best traetment and do not know the adverse drug reactions",]

In [4]:
conv = ABConverter()
## this part is when we want to back from user text to pseudo-sentence
for a in examples_a:
    sentence = conv.a_to_b(a)
    print("A:", a)
    print("B:", sentence, f"(tokens={len(tokenize_b(sentence))})")
    print("-" * 60)

# this part is when we want to back from pseudo-sentence to user text
# examples_b = [
#     "prioritized drugs are: metformin, nilotinib",
#     "TP53 disease_protein MASK indication MASK",
#     "breast cancer indication metformin drug_effect MASK"
# ]
# print("\n=== B -> A ===")
# for b in examples_b:
#     a = conv.b_to_a(b)
#     print("B:", b)
#     print("A:", a)
#     print("-" * 60)

A: for diabetes with egft mutation what is the best traetment and do not know the adverse drug reactions
B: EGFR disease_protein diabetes indication MASK1 drug_effect MASK0 (tokens=7)
------------------------------------------------------------


In [5]:
list_nodes_sentence, node_indices, sentence_indices, mask_index_question = sentence_to_token_id(sentence, mask_token, relation_index)
attr_nodes = node_embedding(list_nodes_sentence)

# ---- Example usage ----
# emb_nodes: [N, D] tensor of your 129k nodes
# node_names: list[str] of length N
# attr_nodes: [Q, D] tensor (your pseudo_sentence embeddings)

all_node_names = nodes['node_name']

index = build_faiss_ip_index(emb_nodes)
hits_per_query = search_topk(index, attr_nodes, all_node_names, k=1)

neighbor_indices = []
neighbors = []
for i, hits in enumerate(hits_per_query):
    print(f"Query {i}")
    for name, cos, nid in hits:
        neighbors.append(name)
        neighbor_indices.append(nid) 
        print(f"  {name}  (id={nid})  cosine={cos:.4f}")

for i, index in zip(node_indices, neighbor_indices):
    sentence_indices[i] = index
    
sentence_indices

relation_token_index_df:    index    relation_name   token
0      1  disease_protein  129381
1      3       indication  129389
2      5      drug_effect  129387
sentence_indices: [129405, 129381, 129405, 129389, 129405, 129387, 129405, 129405, 129405]
remaining_indices: [0, 2, 4, 6, 7, 8]
mask_indices: [6, 4]
remaining_indices:[0, 8, 2, 7]
node_indices: [0, 2]
Sentence embeddings: tensor([[-0.2693,  0.1345,  0.2085,  ..., -0.8011,  0.0349, -0.7274],
        [-0.3037, -0.7627, -0.2508,  ..., -1.0113,  0.8025, -0.9448]])
Query 0
  egfr  (id=125)  cosine=1.0000
Query 1
  diabetes_mellitus_disease  (id=33575)  cosine=0.8221


[125, 129381, 33575, 129389, 129405, 129387, 129405, 129405, 129405]